In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/0_isw_data_collection/1_isw_preprocessed.csv')

### NLTK

In [3]:
#!pip3 install --upgrade pip

In [4]:
#pip install num2words

In [5]:
!pip install scikit-learn

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from num2words import num2words


import nltk
import os
import string
import numpy as np
import copy 
import pickle
import re
import math

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [7]:
data.head()

,Unnamed: 0,date,short_url,title,text_title,full_url,main_html,main_html_v6
0,0,2022-02-25,RusCampaignFeb25,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step..."
1,1,2022-02-26,RusCampaignFeb26,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Katya Stepane..."
2,2,2022-02-27,RusCampaignFeb27,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step..."
3,3,2022-02-28,RusCampaignFeb28,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, Februar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step..."
4,4,2022-03-01,RusCampaignMar1,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, March 1",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Frederick W. Kagan, George Barros, and Katery..."


### Prepare functions

In [8]:
def remove_one_letter_word(data):
    words = word_tokenize(str(data))
    
    new_text = ""
    for w in words:
        if (len(w) > 1):
            new_text = new_text + " " + w
            
    return new_text

In [9]:
#remove_one_letter_word(data['main_html_v6'][0])

In [10]:
def convert_lower_case(data):
    return np.char.lower(data)

In [11]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stop_words)

{'you', 'nor', 'mustn', 'above', 'further', 'couldn', 'me', 'as', "doesn't", "aren't", "couldn't", 'm', 'our', 'had', 'these', 'its', "it's", 'll', 'isn', "you'd", 's', 't', 'them', 'until', 'once', 're', "you're", 'now', 'was', 'mightn', 'has', 'haven', 'their', 'each', 'shouldn', 'yourselves', 'itself', 'an', 'ourselves', 'are', 'been', 'from', 'own', 'too', 'ma', 'and', 'hasn', 'theirs', 'ain', 'in', "you've", 'same', 'the', 'all', 'only', 'o', "wasn't", 'both', 'at', 'have', 'shan', 'yours', 'after', 'to', 'but', 'didn', 'herself', 'while', 'a', 'himself', 'any', 'or', 'just', 'aren', 'is', 'do', "mightn't", 'doesn', 'those', 'that', "weren't", 'through', 'up', 'weren', 'wouldn', 'because', 'we', 'other', "didn't", 'should', "that'll", 'few', 'his', 'were', 'why', 'him', 'having', 'here', 'down', 'y', 'than', "haven't", 'being', 'between', 'will', "she's", 'hers', 'does', 'her', 'not', 'where', 'over', 'then', 'for', 'by', 'ours', "wouldn't", "isn't", "shan't", 'very', 'wasn', "had

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_stop_words = {"no","not"}
    
    stop_words = stop_words - stop_stop_words
    
    words = word_tokenize(str(data))
    
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
            
    return new_text

In [13]:
#remove_stop_words(data['main_html_v6'][0])

In [14]:
def remove_punctuations(data):
    symbols = " !()-[]{};:'\,<>./?@#$%^&*_~ '"""  
    
    for i in range(len(symbols)):
        data = np.char.replace(data,symbols[i]," ")
        data = np.char.replace(data,"  "," ")
    
    data = np.char.replace(data,",", " ")
    
    return data
            

In [15]:
#remove_punctuations(data['main_html_v6'][0])

In [16]:
def stemming(data):
    stemmer = PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
        
    return new_text

def lemmatizing(data):
    lemmatizer = WordNetLemmatizer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + lemmatizer.lemmatize(w)
               
    return new_text

In [17]:
def convert_numbers(data):
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        if w.isdigit():
            if int(w) < 1000000000:
                w = num2words(w)
            else:
                w = " "
        new_text = new_text + " " + w
            
    new_text = np.char.replace(new_text,"-"," ")
        
    return new_text
                
                

In [18]:
#convert_numbers(data['main_html_v6'][0])

In [19]:
def remove_url_from_string(data):
    words = word_tokenize(str(data))
    new_text = ""

    
    for w in words:
        w = re.sub(r'https?:\/\/.*[\r\n]*',"",str(w),flags=re.MULTILINE)
        w = re.sub(r'http?:\/\/.*[\r\n]*',"",str(w),flags=re.MULTILINE)
        
        new_text = new_text + " " + w
        
    return new_text

In [20]:
def preprocess(data, word_root_algo='lemm'):
    
    data = remove_one_letter_word(data)
    data = convert_lower_case(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuations(data)
    data = convert_numbers(data)
    data = remove_url_from_string(data)
    
    if word_root_algo =='lemm':
        data = lemmatizing(data)
    else: 
        data = stemming(data)
        
    
    data = remove_punctuations(data)
    data = remove_stop_words(data)
    
    return data

In [21]:
data['report_text_lemm'] = data['main_html_v6'].apply(lambda x: preprocess(x,"lemm"))

In [22]:
data['report_text_stemm'] = data['main_html_v6'].apply(lambda x: preprocess(x,"stemm"))

In [23]:
data.head()

,Unnamed: 0,date,short_url,title,text_title,full_url,main_html,main_html_v6,report_text_lemm,report_text_stemm
0,0,2022-02-25,RusCampaignFeb25,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step...",mason clark georg barro kateryna stepanenkofe...,mason clark georg barro kateryna stepanenkofe...
1,1,2022-02-26,RusCampaignFeb26,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Katya Stepane...",mason clark georg barro katya stepanenkofebru...,mason clark georg barro katya stepanenkofebru...
2,2,2022-02-27,RusCampaignFeb27,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step...",mason clark georg barro kateryna stepanenkofe...,mason clark georg barro kateryna stepanenkofe...
3,3,2022-02-28,RusCampaignFeb28,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, Februar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step...",mason clark georg barro kateryna stepanenkofe...,mason clark georg barro kateryna stepanenkofe...
4,4,2022-03-01,RusCampaignMar1,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, March 1",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Frederick W. Kagan, George Barros, and Katery...",frederick kagan georg barro kateryna stepanen...,frederick kagan georg barro kateryna stepanen...


In [24]:
data['report_text_lemm'][0]

' mason clark georg barro kateryna stepanenkofebruari twenty five three zero pm estrussian forc enter major ukrainian cities—includ kyiv kherson—for first time februari twenty five russian forc main axe advanc focus kyiv success isol citi bank dnipro river russian militari oper along ukrain northern border le well plan organ conduct eman crimea also le success far diverg perform like aris part differ composit organ russian ground forc element western militari district belaru ukrain north southern militari district black sea fleet south east isw previous observed determin well organ ukrainian resist around kyiv kharkiv also play import role prevent russian militari advanc speed success reportedli planned th russian militari deploy addit forc southeastern belaru like beyond moscow plan use ukrain offset problem challeng russian forc remain much larger capabl ukrain convent militari howev russia like defeat ukrainian regular militari forc secur territori object point come day week putin d

In [25]:
docs = data['report_text_lemm'].tolist()

In [26]:
data.to_csv('data/0_isw_data_collection/2_isw_preprocessed_before_vectosing.csv')

## CountVerctorizer

In [27]:
cv = CountVectorizer(max_df=0.98,min_df=2)
word_count_vector = cv.fit_transform(docs)

word_count_vector.shape

(340, 7401)

In [28]:
word_count_vector

<340x7401 sparse matrix of type '<class 'numpy.int64'>'
	with 213470 stored elements in Compressed Sparse Row format>

In [29]:
with open("models/count_vectorizer_v1.pkl","wb") as handle:
    pickle.dump(cv, handle)

### TF-IDF

In [30]:
tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf = True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [31]:
with open("models/tfidf_transformer_v1.pkl","wb") as handle:
    pickle.dump(tfidf_transformer, handle)

In [32]:
df_idf = pd.DataFrame(tfidf_transformer.idf_, index = cv.get_feature_names_out(),columns=["idf_weights"])
df_idf.sort_values(by=['idf_weights'])

,idf_weights
ukrainian,1.035825
not,1.035825
conduct,1.035825
forc,1.035825
like,1.035825
...,...
murashov,5.733270
murz,5.733270
music,5.733270
monopoli,5.733270


In [33]:
tf_idf_vector = tfidf_transformer.transform(word_count_vector)

In [34]:
tf_idf_vector

<340x7401 sparse matrix of type '<class 'numpy.float64'>'
	with 213470 stored elements in Compressed Sparse Row format>

In [35]:
tf_idf_vector.shape

(340, 7401)

### Transformation with fitted models

In [36]:
tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))

In [37]:
feature_names = cv.get_feature_names_out()

In [38]:
feature_names

array(['00pm', '10', '100', ..., 'zyuganov', 'коридор', 'сухопутный'],
      dtype=object)

In [39]:
def sort_coo(coo_matrix):
    
    tuples = zip(coo_matrix.col, coo_matrix.data)
    
    return sorted(tuples,key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    
    sorted_items = sorted_items[:topn]
    
    score_vals = []
    feature_vals = []
    
    for idx, score in sorted_items:
        
        score_vals.append(round(score,3))
        feature_vals.append(feature_names[idx])
        
    results = {}
    
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]] = score_vals[idx]
    
    return results


def convert_doc_to_vector(doc):
    feature_names = cv.get_feature_names_out()
    top_n = 10
    tf_idf_vector = tfidf.transform(cv.transform([doc]))
    
    sorted_items = sort_coo(tf_idf_vector.tocoo())
    
    keywords = extract_topn_from_vector(feature_names, sorted_items, top_n)
    
    return keywords

In [40]:
data['keywords'] = data['report_text_stemm'].apply(lambda x: convert_doc_to_vector(x))

In [41]:
data['keywords'][0]

{'twenti': 0.693,
 'russian': 0.294,
 'forc': 0.289,
 'februari': 0.205,
 'kyiv': 0.19,
 'ukrainian': 0.177,
 'five': 0.113,
 'envelop': 0.107,
 'bypass': 0.1,
 'time': 0.088}

In [42]:
data_vectorised = tf_idf_vector.toarray()

In [43]:
vectors_df = pd.DataFrame(data_vectorised)

In [44]:
vectors_df.to_csv('data/0_isw_data_collection/3_isw_vectorised.csv')

In [46]:
vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,7391,7392,7393,7394,7395,7396,7397,7398,7399,7400
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.023238,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.033858,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
vectors_df.shape

(340, 7401)